# OpenMC MGXS Library Generation

### Environment

In [ ]:
import math
import numpy as np

import matplotlib.pyplot as plt

import openmc
import openmc.mgxs

import os

# Add path to OpenMC binary
os.environ['PATH'] += r':/path/to/openmc/bin'

# Add location of OpenMC xs data
%env OPENMC_CROSS_SECTIONS=/path/to/openmc/data/endfb-viii.0-hdf5/cross_sections.xml

In [ ]:
model = openmc.Model()

### Materials

In [ ]:
# The resulting MGXS library will have the same name as the material, i.e. UO2.h5
mat1 = openmc.Material(name='UO2')
#mat1.add_element('U', 1.0)
mat1.add_nuclide('U235', 0.20)
mat1.add_nuclide('U238', 0.80)
mat1.add_nuclide('O16', 2.0)
mat1.set_density('g/cm3', 10.0)
model.materials = openmc.Materials([mat1])

### Geometry

In [ ]:
# Create boundary planes
min_x = openmc.XPlane(x0=-1., boundary_type='reflective')
max_x = openmc.XPlane(x0=+1., boundary_type='reflective')
min_y = openmc.YPlane(y0=-1., boundary_type='reflective')
max_y = openmc.YPlane(y0=+1., boundary_type='reflective')
min_z = openmc.ZPlane(z0=-1., boundary_type='reflective')
max_z = openmc.ZPlane(z0=+1., boundary_type='reflective')

# Create a universe an populate it with a single cell
material_universe = openmc.Universe()
material_cell = openmc.Cell()
material_cell.fill = mat1
material_cell.region = +min_x & -max_x & +min_y & -max_y & +min_z & -max_z
material_universe.add_cell(material_cell)

# Create root Cell
root_cell = openmc.Cell(name='root cell')
root_cell.fill = material_universe

# Add boundary planes
root_cell.region = +min_x & -max_x & +min_y & -max_y & +min_z & -max_z

# Create root Universe
root_universe = openmc.Universe(universe_id=0, name='root universe')
root_universe.add_cell(root_cell)

# Create Geometry and set root Universe
model.geometry = openmc.Geometry(root_universe)

### Simulation and Source

Source can be either fixed source or fission.

In [ ]:
# OpenMC simulation parameters
batches = 20
inactive = 10
particles = 1000000

# Instantiate a Settings object
settings = openmc.Settings()
settings.batches = batches
settings.inactive = inactive
settings.particles = particles
settings.output = {'tallies': False}

# Fixed source
#settings.run_mode = 'fixed source'
#source = openmc.IndependentSource()
#bounds = [-1.0, -1.0, -1.0, 1.0, 1.0, 1.0]
#source.space = openmc.stats.Box(bounds[:3], bounds[3:])
#source.angle = openmc.stats.Isotropic()
#source.energy = openmc.stats.Discrete([1.0e+7], [1.0])
#settings.source = source

# Fission source
settings.run_mode = 'eigenvalue'
# Create an initial uniform spatial source distribution over fissionable zones
bounds = [-1.0, -1.0, -1.0, 1.0, 1.0, 1.0]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
settings.source = openmc.Source(space=uniform_dist)

model.settings = settings

### MGXS Library - Group Structure

In [ ]:
# Instantiate an EnergyGroups object
# This is 172 group WIMS
group_edges = np.array([0, 0.003, 0.005, 0.0069, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.042,
                        0.05, 0.058, 0.067, 0.077, 0.08, 0.095, 0.1, 0.115, 0.134, 0.14, 0.16,
                        0.18, 0.189, 0.22, 0.248, 0.28, 0.3, 0.3145, 0.32, 0.35, 0.391, 0.4,
                        0.433, 0.485, 0.5, 0.54, 0.625, 0.705, 0.78, 0.79, 0.85, 0.86, 0.91,
                        0.93, 0.95, 0.972, 0.986, 0.996, 1.02, 1.035, 1.045, 1.071, 1.097,
                        1.11, 1.12535, 1.15, 1.17, 1.235, 1.3, 1.3375, 1.37, 1.44498, 1.475,
                        1.5, 1.59, 1.67, 1.755, 1.84, 1.93, 2.02, 2.1, 2.13, 2.36, 2.55, 2.6,
                        2.72, 2.76792, 3.3, 3.38075, 4, 4.12925, 5.04348, 5.34643, 6.16012,
                        7.52398, 8.31529, 9.18981, 9.90555, 11.2245, 13.7096, 15.9283, 19.4548,
                        22.6033, 24.9805, 27.6077, 30.5113, 33.7201, 37.2665, 40.169, 45.5174,
                        48.2516, 51.578, 55.5951, 67.904, 75.6736, 91.6609, 136.742, 148.625,
                        203.995, 304.325, 371.703, 453.999, 677.287, 748.518, 914.242, 1010.39,
                        1234.1, 1433.82, 1507.33, 2034.68, 2248.67, 3354.63, 3526.62, 5004.5,
                        5530.85, 7465.86, 9118.82, 11137.8, 15034.4, 16615.6, 24787.5, 27394.4,
                        29283, 36978.6, 40867.7, 55165.6, 67379.5, 82297.5, 111090, 122773,
                        183156, 247235, 273237, 301974, 407622, 450492, 497871, 550232, 608101,
                        820850, 907180, 1002590, 1108030, 1224560, 1353350, 1652990, 2018970,
                        2231300, 2465970, 3011940, 3678790, 4493290, 5488120, 6065310, 6703200,
                        8187310, 1e+07, 1.16183e+07, 1.38403e+07, 1.49182e+07, 1.73325e+07, 2e+07])
groups = openmc.mgxs.EnergyGroups(group_edges)

# Initialize MGXS Library for OpenMOC
mgxs_lib = openmc.mgxs.Library(model.geometry)
mgxs_lib.energy_groups = groups

### MGXSLibrary - Scatttering Format and Legendre Order

In [ ]:
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 1

### MGXS Library - Cross Sections

Specify the types of cross sections to compute. In particular, the following are the multi-group cross section `MGXS` subclasses that are mapped to string codes accepted by the `Library` class:

* `TotalXS` (`"total"`)
* `TransportXS` (`"transport"` or `"nu-transport` with `nu` set to `True`)
* `AbsorptionXS` (`"absorption"`)
* `CaptureXS` (`"capture"`)
* `FissionXS` (`"fission"` or `"nu-fission"` with `nu` set to `True`)
* `KappaFissionXS` (`"kappa-fission"`)
* `ScatterXS` (`"scatter"` or `"nu-scatter"` with `nu` set to `True`)
* `ScatterMatrixXS` (`"scatter matrix"` or `"nu-scatter matrix"` with `nu` set to `True`)
* `Chi` (`"chi"`)
* `ChiPrompt` (`"chi prompt"`)
* `InverseVelocity` (`"inverse-velocity"`)
* `PromptNuFissionXS` (`"prompt-nu-fission"`)
* `DelayedNuFissionXS` (`"delayed-nu-fission"`)
* `ChiDelayed` (`"chi-delayed"`)
* `Beta` (`"beta"`)

**Note**: A variety of different approximate transport-corrected total multi-group cross sections (and corresponding scattering matrices) can be found in the literature. At the present time, the `openmc.mgxs` module only supports the `"P0"` transport correction. This correction can be turned on and off through the boolean `Library.correction` property which may take values of `"P0"` (default) or `None`.

In [ ]:
# Specify multi-group cross section types to compute
mgxs_lib.mgxs_types = ['total', 'absorption', 'fission', 'nu-fission', 'chi', 'scatter matrix', 'nu-scatter matrix', 'inverse-velocity']

# Compute cross sections on a nuclide-by-nuclide basis
mgxs_lib.by_nuclide = False

### MGXS Library - Domain

Now we must specify the type of domain over which we would like the `Library` to compute multi-group cross sections. The domain type corresponds to the type of tally filter to be used in the tallies created to compute multi-group cross sections. At the present time, the `Library` supports `"material"`, `"cell"`, `"universe"`, and `"mesh"` domain types. We will use a `"cell"` domain type here to compute cross sections.

**Note:** By default, the `Library` class will instantiate `MGXS` objects for each and every domain (material, cell or universe) in the geometry of interest. However, one may specify a subset of these domains to the `Library.domains` property. In our case, we wish to compute multi-group cross sections in each and every cell since they will be needed in our downstream OpenMOC calculation on the identical combinatorial geometry mesh.

In [ ]:
# Specify a "cell" domain type for the cross section tally filters
mgxs_lib.domain_type = 'cell'

# Specify the cell domains over which to compute multi-group cross sections
mgxs_lib.domains = model.geometry.get_all_material_cells().values()

### MGXS Library - Tallies

Lastly, we use the `Library` to construct the tallies needed to compute all of the requested multi-group cross sections in each domain and nuclide.

In [ ]:
# Construct all tallies needed for the multi-group cross section library
mgxs_lib.build_library()

The tallies can now be export to a "tallies.xml" input file for OpenMC. 

**NOTE**: At this point the `Library` has constructed nearly 100 distinct `Tally` objects. The overhead to tally in OpenMC scales as $O(N)$ for $N$ tallies, which can become a bottleneck for large tally datasets. To compensate for this, the Python API's `Tally`, `Filter` and `Tallies` classes allow for the smart *merging* of tallies when possible. The `Library` class supports this runtime optimization with the use of the optional `merge` paramter (`False` by default) for the `Library.add_to_tallies_file(...)` method, as shown below.

In [ ]:
# Create a "tallies.xml" file for the MGXS Library
tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)

model.tallies = tallies

### Run Simulation

In [ ]:
# Run OpenMC
statepoint_filename = model.run()

### Process Tallies

Our simulation ran successfully and created statepoint and summary output files. We begin our analysis by instantiating a `StatePoint` object. 

In [ ]:
# Load the last statepoint file
sp = openmc.StatePoint(statepoint_filename)

The statepoint is now ready to be analyzed by the `Library`. We simply have to load the tallies from the statepoint into the `Library` and our `MGXS` objects will compute the cross sections for us under-the-hood.

In [ ]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

### Create MGXS Library

In [ ]:
mgxs_file = mgxs_lib.create_mg_library(xs_type='macro')
mgxs_file.export_to_hdf5()
os.rename('mgxs.h5', mat1.name + '.h5')

### Print/Plot Cross Section Data

More xs plotting examples can be found here: https://docs.openmc.org/en/v0.12.0/examples/mgxs-part-ii.html

In [ ]:
# Retrieve the totalXS object from the library
mgxs = mgxs_lib.get_mgxs(material_cell, 'total')
dataframe = mgxs.get_pandas_dataframe(xs_type='macro')
mgxs.print_xs()

# Cast DataFrames as NumPy arrays
y = dataframe['mean'].values
y = np.insert(y, 0, y[0])
x = group_edges

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(8)
ax = fig.add_subplot(111)

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_title('Total Multi-Group Cross Section', fontsize='14')
ax.set_facecolor('white')
ax.plot(x, y)
ax.grid(color='grey')
ax.set_xlabel('eV', fontsize='12')
ax.set_ylabel('cm$^{-1}$', fontsize='12')


### Cleanup

In [ ]:
if os.path.exists('summary.h5'):
    os.remove('summary.h5')
    
if os.path.exists('model.xml'):
    os.remove('model.xml')

if os.path.exists(statepoint_filename):
    os.remove(statepoint_filename)